In [1]:
import os
import torch
import torch.utils.data
import torchvision
from PIL import Image
from pycocotools.coco import COCO
import cv2
import matplotlib.pyplot as plt
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from sklearn.metrics import f1_score, precision_score, recall_score, cohen_kappa_score, classification_report, confusion_matrix
from scipy.stats import rankdata
from numpy import linalg as LA
# import intel_extension_for_pytorch as ipex
from openvino.runtime.ie_api import CompiledModel
from typing import Union
import openvino as ov
import warnings
warnings.filterwarnings("ignore") 
import gradio as gr

%load_ext autotime
# import albumentations as A
# from albumentations.pytorch import ToTensorV2
import numpy as np
device = 'cuda'
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from torch.optim.lr_scheduler import MultiStepLR
import time
import os
plt.style.use('ggplot')
from tqdm import tqdm
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

from custom_utils import (
    Averager, 
    SaveBestModel, 
    save_model, 
    save_loss_plot,
    save_mAP
)

time: 437 ms (started: 2024-05-06 11:15:00 +06:00)


In [2]:
classification_dictionary = {0:'person', 1: 'light', 2:'hydrant', 3:'sign', 4:'bike', 5:'car', 6:'skateboard',
                            7: 'motor', 8: 'bus', 9: 'stroller', 10: 'other', 11: 'truck', 12: 'negative', 13: 'train',
                            14: 'dog', 15: 'scooter'
                            }

time: 0 ns (started: 2024-05-06 11:15:00 +06:00)


In [3]:
transform = {
        'train': torchvision.transforms.Compose([
                        torchvision.transforms.ToPILImage(),

            torchvision.transforms.Grayscale(num_output_channels=1), # Resizing the image as the VGG only take 224 x 244 as input size
            torchvision.transforms.ToTensor(),
#             transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5))
        ])  }

time: 0 ns (started: 2024-05-06 11:15:00 +06:00)


In [4]:
def get_transform():
    custom_transforms = [  torchvision.transforms.Grayscale(num_output_channels=1)# above is for: randomly selecting one for process
        ]
    custom_transforms.append(torchvision.transforms.ToTensor())
    return torchvision.transforms.Compose(custom_transforms)
def get_model_instance_segmentation(num_classes=16):
    # load an instance segmentation model pre-trained pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True, rpn_pre_nms_top_n_test=1000, rpn_post_nms_top_n_test=1000)
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model
model = get_model_instance_segmentation(2)
model
class Custom_predictor(torch.nn.Module):
    def __init__(self,in_channels,num_classes):
        super(Custom_predictor,self).__init__()
        self.cls_score = torch.nn.Linear(in_channels, num_classes)
        self.bbox_pred = torch.nn.Linear(in_channels, num_classes * 4)
        
        
    def forward(self,x):
        if x.dim() == 4:
            torch._assert(
                list(x.shape[2:]) == [1, 1],
                f"x has the wrong shape, expecting the last two dimensions to be [1,1] instead of {list(x.shape[2:])}",
            )
        x = x.flatten(start_dim=1)
#         scores = self.cls_score(x)
        #print(scores.size())
        bbox_deltas = self.bbox_pred(x)
        x = torch.zeros(len(bbox_deltas), 1)
        y = torch.ones(len(bbox_deltas),1)
        if bbox_deltas.is_cuda:
            scores = torch.cat((x, y), 1).to('cuda')
        else:
            scores = torch.cat((x, y), 1).to('cpu')
        return scores, bbox_deltas
in_features = model.roi_heads.box_head.fc7.out_features
#now we can add the custom predictor to the model
num_classes =2
model.roi_heads.box_predictor = Custom_predictor(in_features,num_classes)
checkpoint = torch.load(".\\detector\\best_model.pth")
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()
class Identity(torch.nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x
model.roi_heads.box_predictor.cls_score = Identity()

time: 610 ms (started: 2024-05-06 11:15:00 +06:00)


In [5]:
model

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

time: 0 ns (started: 2024-05-06 11:15:01 +06:00)


In [6]:
int8_ir_path = ".\\classifier\\quantized_model_focal.xml"
core = ov.Core()

model_q = core.read_model(model=int8_ir_path)
model_q = core.compile_model(model=model_q, device_name='CPU')
transform_q = {
        'train': torchvision.transforms.Compose([
            torchvision.transforms.Resize([64,64]), # Resizing the image as the VGG only take 224 x 244 as input size
            torchvision.transforms.RandomHorizontalFlip(), # Flip the data horizontally
            #TODO if it is needed, add the random crop
            torchvision.transforms.RandomHorizontalFlip(p=0.5),
            torchvision.transforms.ToTensor(),
#             transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5))
        ]),
        'test': torchvision.transforms.Compose([
            torchvision.transforms.ToPILImage(),
            torchvision.transforms.Resize([64,64]),
            torchvision.transforms.ToTensor(),   ])
    }

time: 172 ms (started: 2024-05-06 11:15:01 +06:00)


In [49]:
def sepia(inp):
    inp = transform['train'](inp)
    model.eval()
    model.to('cuda')


    image = inp.unsqueeze(0)
    image = image.to('cuda')

    box = model(image)
    boxes = box[0]['boxes']
    boxes
    img_gray = np.float64(image.to('cpu'))
    img_gray= img_gray[0]
    img = img_gray
    img = cv2.merge([img_gray[0],img_gray[0],img_gray[0]])
    for i in boxes:
        xmin, ymin, xmax, ymax = i
        pt1 = (int(xmin), int(ymin))
        pt2 = (int(xmax), int(ymax))
    #     crop = img.permute(1, 2, 0).numpy()
        crop = img[int(ymin):int(ymax), int(xmin):int(xmax)]
        crop = transform_q['test'](crop)
        crop = crop.unsqueeze(0)
        output_layer = model_q.output(0)
        output = model_q(crop)[output_layer]
        output_batch = torch.from_numpy(output)
        output_batch = torch.nn.functional.softmax(output_batch)
        _, predicted = output_batch.max(1)
        if predicted != 12:
            bnd_img = cv2.rectangle(img,pt1, pt2,(53,255, 130),1)
            bnd_img = cv2.putText(
                bnd_img,
                str(classification_dictionary[int(predicted)]),
                (int(xmin)-5, int(ymin)),
                fontFace = cv2.FONT_HERSHEY_SIMPLEX,
                fontScale = 0.5,
                color = (78, 0, 78),
                thickness=2)
    bnd_img = (bnd_img * 255).astype(np.uint8)
    return bnd_img

time: 0 ns (started: 2024-05-06 11:25:50 +06:00)


In [50]:
# demo = gr.Interface(
#     fn=predict,
#     inputs=[gr.components.Image(label='Input', type='pil'),
#             gr.components.Number(label='Number of variations')],
#     outputs=gr.components.Gallery(label='Variations'),
#     allow_flagging='never'
# )

time: 0 ns (started: 2024-05-06 11:25:51 +06:00)


In [51]:
import numpy as np
import gradio as gr



demo = gr.Interface(sepia, gr.Image(), "image")
if __name__ == "__main__":
    demo.launch()

Running on local URL:  http://127.0.0.1:7874

To create a public link, set `share=True` in `launch()`.


time: 204 ms (started: 2024-05-06 11:25:51 +06:00)
